# Read SIFT 1 million 

In [1]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays

In [2]:
Sys.cpu_info()[1].model

"Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz"

In [3]:
path = joinpath(homedir(), "Datasets", "SIFT1M",
                "sift-128-euclidean.hdf5")

"/Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [4]:
f = h5open(path, "r")

🗂️ HDF5.File: (read-only) /Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5
├─ 🏷️ distance
├─ 🔢 distances
├─ 🔢 neighbors
├─ 🔢 test
└─ 🔢 train

In [5]:
X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
neighbors = read(f["neighbors"])
distances = read(f["distances"])

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(neighbors)
@show size(distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(neighbors) = (100, 10000)
size(distances) = (100, 10000)


(100, 10000)

## Clustering.jl

Create the pqcodes for the training points

In [6]:
n_features = size(X_tr_vecs,1)
n_features

128

In [7]:
n_clusters = 16
R_per_feature = []
@showprogress for j in  1:n_features
    R = kmeans(X_tr_vecs[[j],:], n_clusters; maxiter=200)
    push!(R_per_feature, R)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:33


In [42]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[24.323595, 0.7822317, 72.72988, 50.838577, 118.64538, 85.2806, 32.818943, 128.41817, 11.905438, 60.784718, 17.379864, 96.78006, 140.06845, 41.85292, 108.1372, 6.282033]
(16, 128)


In [110]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[24.323595, 0.7822317, 72.72988, 50.838577, 118.64538, 85.2806, 32.818943, 128.41817, 11.905438, 60.784718, 17.379864, 96.78006, 140.06845, 41.85292, 108.1372, 6.282033]
(16, 128)


In [66]:
@benchmark P = hcat([r.centers[:] for r in R_per_feature]...)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.082 μs … 197.672 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.704 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.305 μs ±   9.938 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▆▄▄▄▄▃▂▂▂▂▂▁▁▁                                             ▁
  ████████████████████▇█▇▇▇▇▇▆▆▇▇▇▆▇█▇▇▇▇█▇▇▆▇▆▆▆▆▅▆▅▅▄▄▄▅▄▄▄▅ █
  13.1 μs       Histogram: log(frequency) by time      60.7 μs <

 Memory estimate: 28.47 KiB, allocs estimate: 137.

Another way to construct the matrix would be

In [105]:
function build_prototype_matrix(R, n_clusters, n_features)
    
    prototypes = Array{Float32}(undef, n_clusters, n_features);
    for j in 1:n_features
        prototypes[:,j] .= vec(R_per_feature[j].centers)
    end
    return prototypes
end

build_prototype_matrix (generic function with 1 method)

In [107]:
@benchmark P = build_prototype_matrix(R_per_feature, n_clusters, n_features)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  61.832 μs … 422.744 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     65.480 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   71.425 μs ±  15.373 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▇█▇▆▅▄▂▂           ▁▁▂▂▂▂▂▁▂▂▁                              ▂
  ██████████▇▅▅▅▅▆▆▆███████████████▇▇▆▅▅▅▅▅▅▆▇▇▇█████▇▇█▇▆▅▆▆▄ █
  61.8 μs       Histogram: log(frequency) by time       129 μs <

 Memory estimate: 44.12 KiB, allocs estimate: 897.

In [112]:
P2 = build_prototype_matrix(R_per_feature, n_clusters, n_features);

In [114]:
isapprox(P, P2)

true

We can generate the PQcodes for the data as follows:

In [9]:
PQcodes = hcat([Int8.(r.assignments[:]) for r in R_per_feature]...)';

In [10]:
size(PQcodes)

(128, 1000000)

Here each vector prototype is in fact a single scalar (vecause subvectors have a single coordinate)

In [115]:
size(P)

(16, 128)

We need a method to, given a vector and a collection of prototypes per feature, find which are the closest

In [190]:
query = X_te_vecs[:,1];

In [261]:
closest_prototypes = Array{UInt8}(undef, n_features, 1);

In [271]:
typeof(typemax(Float32))

Float32

In [322]:
@inline function seuclidean(x::T,y::T) where {T}
    aux::T = (x -y)
    return aux * aux
end

function find_closest_coordinate(dist, 
                                 value::T,
                                 vector) where T
    best_coordinate = 1
    min_distance::T = typemax(T)
    for (j,x) in enumerate(vector)
        current_dist = dist(x, value)
        if current_dist < min_distance
            best_coordinate = j
            min_distance = current_dist
        end
    end
    return best_coordinate
end

find_closest_coordinate (generic function with 3 methods)

In [323]:
find_closest_coordinate(seuclidean, Float32(48.), query)
print(query)

Float32[1.0, 3.0, 11.0, 110.0, 62.0, 22.0, 4.0, 0.0, 43.0, 21.0, 22.0, 18.0, 6.0, 28.0, 64.0, 9.0, 11.0, 1.0, 0.0, 0.0, 1.0, 40.0, 101.0, 21.0, 20.0, 2.0, 4.0, 2.0, 2.0, 9.0, 18.0, 35.0, 1.0, 1.0, 7.0, 25.0, 108.0, 116.0, 63.0, 2.0, 0.0, 0.0, 11.0, 74.0, 40.0, 101.0, 116.0, 3.0, 33.0, 1.0, 1.0, 11.0, 14.0, 18.0, 116.0, 116.0, 68.0, 12.0, 5.0, 4.0, 2.0, 2.0, 9.0, 102.0, 17.0, 3.0, 10.0, 18.0, 8.0, 15.0, 67.0, 63.0, 15.0, 0.0, 14.0, 116.0, 80.0, 0.0, 2.0, 22.0, 96.0, 37.0, 28.0, 88.0, 43.0, 1.0, 4.0, 18.0, 116.0, 51.0, 5.0, 11.0, 32.0, 14.0, 8.0, 23.0, 44.0, 17.0, 12.0, 9.0, 0.0, 0.0, 19.0, 37.0, 85.0, 18.0, 16.0, 104.0, 22.0, 6.0, 2.0, 26.0, 12.0, 58.0, 67.0, 82.0, 25.0, 12.0, 2.0, 2.0, 25.0, 18.0, 8.0, 2.0, 19.0, 42.0, 48.0, 11.0]

In [324]:
@benchmark find_closest_coordinate(seuclidean, Float32(48.), query)


BenchmarkTools.Trial: 10000 samples with 755 evaluations.
 Range (min … max):  160.910 ns … 693.626 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     169.713 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   176.307 ns ±  22.495 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▄ ▅█▅▃▂▂▂▁ ▁                                                 ▁
  ███████████████████▇████▇▇█▇▇▇▇▇▆▇▇▇▆▆▆▆▆▇▆▆▆▅▅▅▆▅▄▄▆▆▄▅▅▄▄▄▅ █
  161 ns        Histogram: log(frequency) by time        275 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [325]:
@assert 3 == find_closest_coordinate(seuclidean, 2., [3.,5,2,1])

@assert 4 == find_closest_coordinate(seuclidean, 1., [3.,5,2,1])

In [342]:
function encode(dist, vector, prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    @inbounds for (j,vector_x) in enumerate(vector)
        for prototypes_j in eachcol(prototypes)
            closest_prototypes[j] = find_closest_coordinate(dist, vector_x, prototypes_j)
        end
    end
    return closest_prototypes
end

encode (generic function with 2 methods)

In [346]:
@benchmark encode(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  238.500 μs …  1.962 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     268.101 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   300.961 μs ± 66.268 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      █▆▅▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁ ▁                     ▂
  ▇▃▁▁█████████████████████████████████████████████▇▇▇▇▇▅▆▆▄▆▆ █
  238 μs        Histogram: log(frequency) by time       522 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [344]:
size(P)

(16, 128)

In [350]:
function compute_ADC(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            ADC_table[p,j] = dist(query[j], @view prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC (generic function with 1 method)

In [351]:
size(P)

(16, 128)

In [352]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [353]:
@benchmark compute_ADC(query, P, SEuclidean0)

BenchmarkTools.Trial: 5805 samples with 1 evaluation.
 Range (min … max):  634.068 μs … 231.021 ms  ┊ GC (min … max): 0.00% … 99.60%
 Time  (median):     733.786 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   854.304 μs ±   3.030 ms  ┊ GC (mean ± σ):  4.64% ±  1.31%

   █▅▃▂                                                          
  ▇█████▇▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  634 μs           Histogram: frequency by time         1.79 ms <

 Memory estimate: 272.25 KiB, allocs estimate: 10501.

In [354]:
adc_table = compute_ADC(query, P, SEuclidean0);

In [355]:
function adc_dist(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @fastmath for j in eachindex(x_code)
        res+= adc_table[x_code[j], j]
    end
    return res
end

adc_dist (generic function with 1 method)

In [362]:
query_code = Int8.(rand(1:16,128));
query_code = encode(sqeuclidean, query, P)
x_code = PQcodes[:,1];
adc_table = compute_ADC(query, P, SEuclidean0);
adc_dist(query_code, x_code,  adc_table)

525519.75f0

In [ ]:
@benchmark adc_dist(query_code, x_code,  adc_table)

In [ ]:
x = X_tr_vecs[:,1];
@benchmark SEuclidean0(x, query)

## Euclidean  Squared Distance

In [6]:
function SEuclidean(X, query)
    d = (query .- X) .* (query .- X)
    return sum(d, dims=1)
end

SEuclidean (generic function with 1 method)

In [221]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

SEuclidean_2 (generic function with 1 method)

In [8]:
query = X_te_vecs[:,1];

In [9]:
@benchmark SEuclidean(X_te_vecs, query)

BenchmarkTools.Trial: 1078 samples with 1 evaluation.
 Range (min … max):  1.807 ms … 38.863 ms  ┊ GC (min … max):  0.00% … 88.81%
 Time  (median):     4.323 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.622 ms ±  4.211 ms  ┊ GC (mean ± σ):  11.52% ± 11.30%

  ▄▂ ▆█▃                                                      
  ██▇███▆▇▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▄▄▁▄▄▄ ▇
  1.81 ms      Histogram: log(frequency) by time     36.4 ms <

 Memory estimate: 4.92 MiB, allocs estimate: 4.

In [10]:
@benchmark SEuclidean_2(X_te_vecs, query)

BenchmarkTools.Trial: 3167 samples with 1 evaluation.
 Range (min … max):  1.314 ms …   2.442 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.538 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.560 ms ± 156.194 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▅▂▃▃▂▄▁▃▂▃▄▄▆▆▇▆▅▃▅█▅▃▃▃▂                                  
  ▁▄█████████████████████████▇█▆▅▆▆▄▄▄▄▃▃▄▃▂▂▃▂▃▃▂▂▂▂▃▂▂▂▂▁▂▁ ▄
  1.31 ms         Histogram: frequency by time        2.04 ms <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [11]:
@benchmark SEuclidean(X_tr_vecs, query)

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  432.581 ms … 525.352 ms  ┊ GC (min … max): 0.19% … 14.88%
 Time  (median):     454.724 ms               ┊ GC (median):    0.18%
 Time  (mean ± σ):   472.292 ms ±  36.352 ms  ┊ GC (mean ± σ):  6.43% ±  7.06%

  █ ██  █    █  █               █               █       ██    █  
  █▁██▁▁█▁▁▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁██▁▁▁▁█ ▁
  433 ms           Histogram: frequency by time          525 ms <

 Memory estimate: 492.10 MiB, allocs estimate: 4.

In [12]:
@benchmark SEuclidean_2(X_tr_vecs, query)

BenchmarkTools.Trial: 34 samples with 1 evaluation.
 Range (min … max):  141.612 ms … 163.347 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     151.451 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   151.293 ms ±   4.372 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▁      ▄   ▁█   ▄                              
  ▆▁▁▁▁▁▁▁▁▆▆▁▆▆▆▁█▁▁▆▆▁▁█▆▆▆██▁▆▁█▁▁▆▁▆▁▆▁▆▆▁▆▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  142 ms           Histogram: frequency by time          163 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

## Finding top k distances (and their ids)


The first naive thing we can do consist on computing all distances and then sorting them to get the top k closest vectors to the query vector

In [13]:
function top_k_ids(X, query)
    distances = SEuclidean_2(X, query)
    top_k_indices = sortperm(distances)
    return top_k_indices
end

top_k_ids (generic function with 1 method)

In [14]:
@benchmark top_k_ids(X_te_vecs, query)[1:10]

BenchmarkTools.Trial: 2144 samples with 1 evaluation.
 Range (min … max):  1.949 ms …  38.365 ms  ┊ GC (min … max): 0.00% … 93.95%
 Time  (median):     2.256 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.313 ms ± 807.566 μs  ┊ GC (mean ± σ):  0.73% ±  2.03%

          ▂▄▃▇▅▆█▄▆▆▃▄▃ ▃▁                                     
  ▅▅▇▆▆▇▇█████████████████▇██▇▆▆▇▆▄▅▄▄▃▂▅▂▃▂▃▂▂▂▂▂▂▁▂▁▂▁▁▁▂▂▁ ▄
  1.95 ms         Histogram: frequency by time        3.01 ms <

 Memory estimate: 156.50 KiB, allocs estimate: 6.

A slightly better approach consist on using `partialsortperm` to simply sort a subset of the distances vector.

In [15]:
function top_k_ids_2(X, query, k)
    distances = SEuclidean_2(X, query)
    top_k_indices = partialsortperm(distances, 1:k)
    return top_k_indices
end

top_k_ids_2 (generic function with 1 method)

In [16]:
@benchmark top_k_ids_2(X_te_vecs, query, 10)

BenchmarkTools.Trial: 3010 samples with 1 evaluation.
 Range (min … max):  1.340 ms …  39.912 ms  ┊ GC (min … max): 0.00% … 95.67%
 Time  (median):     1.632 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.647 ms ± 712.190 μs  ┊ GC (mean ± σ):  0.77% ±  1.74%

        ▃▁▂       ▁  ▁▅▅▂▃▃█▅▃▂▁▁▂▁                            
  ▂▁▁▁▃▇█████▆▇▇▇▇█▇█████████████████▇▅▆▅▅▄▅▄▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▃ ▅
  1.34 ms         Histogram: frequency by time        2.04 ms <

 Memory estimate: 156.44 KiB, allocs estimate: 7.

### Storing top k distances in a priority queue

A better alternative consists on using a priority queue. This. queue will keep only k distances in memory (no need to store all distances between the query point and all possible candidates).

In [17]:
sort!([1,54,3,24,10])

5-element Vector{Int64}:
  1
  3
 10
 24
 54

In [18]:
a = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [19]:
function SEuclidean_3(X, query, top_k)
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end
            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end

    end
    return result
end

SEuclidean_3 (generic function with 1 method)

In [20]:
@benchmark SEuclidean_3(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 33 samples with 1 evaluation.
 Range (min … max):  143.842 ms … 158.893 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     153.199 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   152.719 ms ±   3.664 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃                          █                 ▃ ▃▃   ▃          
  █▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▇▇▁▁▁▇▁▁▁▇█▇▁▁▁▇▇▇▇▇▇▇▇▁▁▇▇▁█▇██▁▇▁█▁▁▁▇▁▁▁▇ ▁
  144 ms           Histogram: frequency by time          159 ms <

 Memory estimate: 16.67 KiB, allocs estimate: 120.

In [21]:
@benchmark top_k_ids_2(X_tr_vecs, query, 10)

BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  155.453 ms … 199.189 ms  ┊ GC (min … max): 0.00% … 15.91%
 Time  (median):     168.506 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   168.850 ms ±   7.160 ms  ┊ GC (mean ± σ):  0.63% ±  2.90%

            ▄  ▁  ▁ ▄█ ▄      ▁                                  
  ▆▁▁▁▁▁▁▁▁▆█▆▆█▆▆█▆██▆█▆▁▁▁▁▆█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  155 ms           Histogram: frequency by time          199 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 7.

In [22]:
function SEuclidean_4(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    @inbounds @fastmath for m in top_k:n_examples
        res = zero(eltype(X))
        @simd for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end


SEuclidean_4 (generic function with 1 method)

In [23]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10) 

BenchmarkTools.Trial: 158 samples with 1 evaluation.
 Range (min … max):  28.027 ms … 35.786 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     32.131 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   31.630 ms ±  1.654 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                       ▃▆▃ ▄█▁                 
  ▄▃▃▆▃▄▁▄█▆▆▄▄▃▁▃▃▃▇▃▄▄▃▇▁▁▁▄▇▄▃▄▃▄▃▆████▄███▃▇█▆█▆▆▁▇▃▄▁▁▁▄ ▃
  28 ms           Histogram: frequency by time        34.4 ms <

 Memory estimate: 15.97 KiB, allocs estimate: 114.

In [24]:
X_tr_200k = X_tr_vecs[:,1:200_000]
@benchmark SEuclidean_4(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 803 samples with 1 evaluation.
 Range (min … max):  5.484 ms …   7.608 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.305 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.214 ms ± 409.202 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂█▆                       ▂  ▂▃▂▃▄▃▁▅    ▁                  
  ▆███▆▆▃▂▂▂▂▃▄▃▂▃▃▄▃▃▄▄▇▄▅▇▆█▅███████████▆██▃▅▅▄▄▃▂▅▂▂▂▃▁▁▂▁ ▄
  5.48 ms         Histogram: frequency by time        7.04 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

In [25]:
SEuclidean_4(X_tr_vecs, query, 10)

10-element Vector{Float64}:
 54229.0
 55091.0
 59531.0
 65260.0
 65697.0
 67010.0
 68247.0
 69844.0
 71441.0
 71861.0

In [26]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        @turbo  for j in 1:n_features
            aux = (query[j] - X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

SEuclidean_5 (generic function with 1 method)

In [27]:
@benchmark SEuclidean_5(X_tr_200k, query, 10) 

BenchmarkTools.Trial: 727 samples with 1 evaluation.
 Range (min … max):  5.698 ms …   9.772 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.787 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.854 ms ± 651.729 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▅ ▂▃▄▅▆▄▅█▄▅▇▂▂▁                                   
  ▄▄▄█▆▅█▇▇▆████████████████▇▄▇▅▅▄▄▄▁▂▃▃▃▃▄▃▂▃▁▃▃▃▃▁▂▃▂▂▃▁▃▁▂ ▄
  5.7 ms          Histogram: frequency by time        9.21 ms <

 Memory estimate: 13.94 KiB, allocs estimate: 94.

# Comparison with Distances.jl


Distances.jl provides the method 'pairwise' to compute pairwise distances between two matrices.

In [28]:
@show typeof(query)
@show size(query)
@show typeof(X_tr_vecs)
@show size(X_tr_vecs)

typeof(query) = Vector{Float32}
size(query) = (128,)
typeof(X_tr_vecs) = Matrix{Float32}
size(X_tr_vecs) = (128, 1000000)


(128, 1000000)

In [29]:
v1 = [1,2,3]
v2 = [0,0,1]

y = [0,0,2]
X = [v1 v2]

3×2 Matrix{Int64}:
 1  0
 2  0
 3  1

In [30]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_5(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        dist = zero(eltype(X))
        @turbo  for j in 1:n_features
            dist = (query[j] - X[j,m])
            dist += aux * aux
        end
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

SEuclidean_5 (generic function with 1 method)

In [31]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(eltype(X), n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

SEuclidean_2 (generic function with 1 method)

In [137]:
y_colvec = [y y][:,[1]]
query_mat = [query query][:,[1]];

LoadError: UndefVarError: y not defined

In [33]:
SEuclidean_2(X, y)

2-element Vector{Int64}:
 6
 1

In [34]:
pairwise(SqEuclidean(), y_colvec, X)

1×2 Matrix{Int64}:
 6  1

In [36]:
@benchmark pairwise(SqEuclidean(), query_mat, X_tr_vecs)

BenchmarkTools.Trial: 55 samples with 1 evaluation.
 Range (min … max):  82.712 ms … 124.640 ms  ┊ GC (min … max): 0.00% … 32.03%
 Time  (median):     90.381 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   92.336 ms ±   8.605 ms  ┊ GC (mean ± σ):  0.79% ±  4.32%

      ▂█▅▂            ▂                                         
  ▅█▁▅█████▁█▅▅▁█▁▁▅▁███▅▁▁▅▁▅▁█▅▅▁▁▅▅▁▅▁▅▁▁▁▅▁▁▁▁▁▅▁█▅▅▁▅▁▁▁▅ ▁
  82.7 ms         Histogram: frequency by time          109 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 6.

In [37]:
@benchmark SEuclidean_2(X_tr_vecs, query)

BenchmarkTools.Trial: 34 samples with 1 evaluation.
 Range (min … max):  138.252 ms … 154.839 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     150.056 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   148.211 ms ±   4.644 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                       ▁      █      █▄▁         
  ▆▁▁▁▁▁▆▁▁▆▁▁▆▁▆▆▆▁▆▁▆▁▁▆▁▁▁▆▆▆▁▁▁▁▁▁▁█▁▁▁▆▆▁█▁▆▁▁▆▁███▁▁▆▁▁▁▆ ▁
  138 ms           Histogram: frequency by time          155 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

In [38]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10)

BenchmarkTools.Trial: 156 samples with 1 evaluation.
 Range (min … max):  28.160 ms … 34.641 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     32.683 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   32.185 ms ±  1.529 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                           ▂ ▅▃█ ▄▅▃▂ ▃ ▂      
  ▇▁▅▃▃▃▅▁▃▃▁▁▃▁▃▅▃▃▁▃▆▁▇▅▁▅▃▃▁▅▁▅▁▅▁▃▅▃▇▆▇██████████▅█▅█▆▇▃▅ ▃
  28.2 ms         Histogram: frequency by time        34.1 ms <

 Memory estimate: 14.06 KiB, allocs estimate: 114.

We can verify that the sorted distances by `SEuclidean_4` are the same as the results provided with `pairwise` from `Distances.jl`

In [39]:
res2 = SEuclidean_4(X_tr_vecs, query, 10);
res = pairwise(SqEuclidean(), query_mat, X_tr_vecs)
res = sort(res, dims=2)[1:10]
res == res2

true

## Parallel implementation

# SIMD tests

In [40]:

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

find_val_in_array_simd (generic function with 1 method)

In [41]:
function jdotavx(a, b)
    s = zero(eltype(a))
    @turbo for i ∈ eachindex(a, b)
        s += a[i] * b[i]
    end
    s
end

jdotavx (generic function with 1 method)